In [ ]:
# ! pip install datasets transformers[sentencepiece] sacrebleu peft -q

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

In [2]:
device = "cuda"

In [3]:
data = pd.read_json("../data/datafinal.json")

In [4]:
mlb = MultiLabelBinarizer()
mlb.fit(data["CodeList"])

MultiLabelBinarizer()

In [5]:
test_data = data[~data["ContainsCode"].apply(lambda x: isinstance(x, bool))].copy()
train_data = data[data["ContainsCode"].apply(lambda x: isinstance(x, bool))].copy()

In [7]:
model_checkpoint = "../codet5-basenew_model/checkpoint-4200"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
for t in mlb.classes_:
    if t != tokenizer.decode(tokenizer(t)["input_ids"], skip_special_tokens=True):
        print("tokenizer not supported")
        break

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.to(device)

In [12]:
# i = 90
i = 49
text = test_data["Text"].iloc[i]
# text = """Software development has changed significantly over the years. Nowadays, it's common to use high-level programming languages like JavaScript, which are more developer-friendly. A simple JavaScript code to display a message in the console would be:\n\nconsole.log('Hello, World!');\n\nDespite their simplicity, high-level languages are powerful tools for creating complex applications."""
answer = test_data["CodeList"].iloc[i]

print(text)
print("===========")
print(answer)

Software development requires a good understanding of algorithms and data structures. These are fundamental concepts that every developer should be familiar with. Algorithms are sets of instructions for solving a problem or achieving a goal, while data structures are ways of organizing and storing data. Here's a simple example of a sorting algorithm in JavaScript.
 function bubbleSort(arr) {
 for(let i = 0; i < arr.length; i++) {
 for(let j = 0; j < arr.length - i - 1; j++) {
 if(arr[j] > arr[j + 1]) {
 let temp = arr[j];
 arr[j] = arr[j + 1];
 arr[j + 1] = temp;
 }
 }
 }
 return arr;
 }
 This is a bubble sort algorithm, which sorts an array in ascending order.



In [13]:
output = model.generate(**tokenizer.encode_plus(text, return_tensors="pt").to(device), max_length=256)

In [14]:
print(tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0].replace('\\n', '\n').replace('\\n', '\n'))

function bubbleSort(arr) {
 for(let i = 0; i < arr.length; i++) {
 for(let j = 0; j < arr.length - i - 1; j++) {
 if(arr[j] > arr[j + 1]) {
 let temp = arr[j];
 arr[j] = arr[j + 1];
 arr[j + 1] = temp;
 }
 }
 return arr;


In [15]:
len(mlb.classes_)

93

In [16]:
count = 0
y_test = []
y_pred = []
for _, row in tqdm(train_data.iterrows(), total=len(train_data)):
    output = model.generate(**tokenizer.encode_plus(row["Text"], return_tensors="pt").to(device), max_length=256)
    output = tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
    x1 = mlb.transform([row["CodeList"].replace('\n', '\\n').replace('\t', '\\t')])
    x2 = mlb.transform([output.replace('\n', '\\n').replace('\t', '\\t')])
    y_test.append(row["CodeList"].replace('\n', '\\n').replace('\t', '\\t'))
    y_pred.append(output.replace('\n', '\\n').replace('\t', '\\t'))
    if abs(x1-x2).sum() != 0:
        print(row["Text"])
        print("===============")
        print(row["CodeList"])
        print("++++++++++++")
        print(output)
        print("----------------"*100)
        count += 1

  0%|          | 0/1377 [00:00<?, ?it/s]

Software development is a creative process. It's about coming up with solutions to problems, and then implementing those solutions in code. This requires a good understanding of the problem domain, as well as the ability to think logically and systematically. It also requires a good understanding of the tools and languages you're using. For instance, if you're using JavaScript, you'll need to understand concepts like variables, functions, and closures. A simple JavaScript function to add two numbers might look like function add x, y, return x + y.
function add(x, y)', 'return x + y
++++++++++++
function add x, y, return x + y
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
accuracy_score(mlb.transform(y_test), mlb.transform(y_pred))

0.9789397240377633

In [18]:
count

29